## 🎯 Exercise: Build an Image Generation Agent with Cost Approval

**The scenario:**

Build an agent that generates images using the MCP server, but requires approval for "bulk" image generation:
- Single image request (1 image): Auto-approve, generate immediately
- Bulk request (>1 image): Pause and ask for approval before generating multiple images
- Explore different publicly available Image Generation MCP Servers

In [34]:
## Initial Set-up : Configure Gemini API

import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [35]:
## Import ADK components

import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

## Configure Retry Options

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print("✅ Retry options configured successfully.")


✅ ADK components imported successfully.
✅ Retry options configured successfully.


In [36]:
## MCP 
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [37]:
# This function mirrors the shipping example. 
# It auto-approves single-image requests, and
# It requests confirmation from the human for bulk requests (>1 images). 

BULK_THRESHOLD = 1  # >1 images means bulk

def generate_images_with_approval(prompt: str, num_images: int, tool_context: ToolContext) -> dict:
    """MCP-backed image generation tool with approval for bulk requests.

    Args:
        prompt: Text prompt for image generation
        num_images: Number of images to generate
        tool_context: ADK-provided ToolContext for pause/resume

    Returns:
        dict with status and (if approved) base64 image payload(s)
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Single image request (1 image): Auto-approve, generate immediately
    if num_images <= BULK_THRESHOLD:
        return {
            "status": "approved",
            "num_images": num_images,
            "prompt": prompt,
            "message": f"Auto-approved generation of {num_images} image(s).",
        }
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: Bulk request (>1 image): Pause and ask for approval before generating multiple images
    
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=(
                f"⚠️ Bulk image generation requested: {num_images} images for prompt: '{prompt}'. "
                "This may incur higher cost. Approve?"
            ),
            payload={"prompt": prompt, "num_images": num_images},
        )
        return {
            "status": "pending",
            "message": f"Request to generate {num_images} images requires approval",
        }
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    # Resumed call: handle human decision
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "num_images": num_images,
            "prompt": prompt,
            "message": f"Human approved {num_images} images.",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Human rejected generation of {num_images} images.",
        }

print("✅ Long-running functions created!")

✅ Long-running functions created!


In [47]:
# Create image generation agent with pausable tool
image_agent = LlmAgent(
    name="image_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are an image generation tool coordinator.
  
  When users request to generate images:
   1. Use the generate_images_with_approval tool with the number of images to create
   2. Auto approve if requested one image generation and send for human approval if requested to generate >1 image
   3. If the image generation status is 'pending', inform the user that approval is required
   4. After receiving the final result, provide a clear summary including:
      - Number of images requested to generate
      - image generation status (approved/rejected)
   5. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=generate_images_with_approval)],
)

print("✅ Image generation Agent created!")

✅ Image generation Agent created!


In [48]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
image_generation_app = App(
    name="image_generation_coordinator",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


/tmp/ipykernel_47/3026063764.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [49]:
session_service = InMemorySessionService()

# Create runner with the resumable app
image_runner = Runner(
    app=image_generation_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


---

**✅ Recap: Your pausable shipping agent is now complete!**

You've created:
1. ✅ A tool that can pause for approval (`generate_images_with_approval`)
2. ✅ An agent that uses this tool (`image_generation_agent`)
3. ✅ A resumable app that saves state (`image_generation_app`)
4. ✅ A runner that can handle pause/resume (`image_runner`)

**Next step:** Build the workflow code and test that our Agent detects pauses and handles approvals.

---

In [50]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

In [51]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

In [52]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


In [53]:
async def run_image_generation_workflow(query: str, num_images: int):
    """Runs a image generation workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve >1 image generation (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Determine auto-approval dynamically
    auto_approve = (num_images == 1)

    # Generate unique session ID
    session_id = f"image_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="image_generation_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_images > 1, the Agent returns the special `adk_request_confirmation` event
    async for event in image_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...(num_images={num_images})...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in image_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")

✅ Workflow function ready


In [55]:
# Demo 1: Generate one image. Agent receives auto-approved status from tool
await run_image_generation_workflow("Generate 1 image showing dog", 1)

# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_image_generation_workflow("Generate 5 images including dogs playing in a farm near mountains and river", 5)


User > Generate 1 image showing dog





User > Generate 5 images including dogs playing in a farm near mountains and river



⏸️  Pausing for approval...(num_images=5)...
🤔 Human Decision: REJECT ❌

Agent > The generation of 5 images has been rejected by a human.

